In [1]:
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import json
import os

In [23]:
#Initialize Variables
project = "cloudrun-test-415115"
location="us-central1"
sentence_file_path = "D:\Projects\Dementia-Hackathon\Files\dementia_sentences.json"
index_name="dementia_index" #Get this from the console or the previous step
index_endpoint = "4083089206304309248"

In [20]:
#Initialize the Model
aiplatform.init(project=project,location=location)
vertexai.init()
model = GenerativeModel("gemini-pro")
dt_index_ep = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name=index_endpoint)

In [21]:
#Generate Embeddings

def generate_text_embeddings(sentences) -> list:    
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    embeddings = model.get_embeddings(sentences)
    vectors = [embedding.values for embedding in embeddings]
    return vectors

In [22]:
#Generate Context

def generate_context(ids,data):
    concatenated_names = ''
    for id in ids:
        for entry in data:
            if entry['id'] == id:
                concatenated_names += entry['sentence'] + "\n" 
    return concatenated_names.strip()

In [26]:
def load_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            entry = json.loads(line)
            data.append(entry)
    return data

In [27]:
data=load_file(sentence_file_path)

In [50]:
query=["What are the support available for Dementia"]
qry_emb=generate_text_embeddings(query)

In [51]:
response = dt_index_ep.find_neighbors(
    deployed_index_id = index_name,
    queries = [qry_emb[0]],
    num_neighbors = 10
)

In [52]:
matching_ids = [neighbor.id for sublist in response for neighbor in sublist]
context = generate_context(matching_ids,data)

In [53]:
prompt=f"Based on the context delimited in backticks, answer the query. ```{context}``` {query}"

In [54]:
chat = model.start_chat(history=[])
response = chat.send_message(prompt)

In [55]:
print(response.text)

## Support Available for People with Dementia

There's a range of support available for people with dementia and their families, both at home and in the community. Here are some key points:

**At Home:**

* **Family and friends:** Often the first and most important source of support.
* **Homecare workers/personal assistants:** Help with tasks like cooking, cleaning, and personal care.
* **Respite care:** Offers temporary breaks for family caregivers.
* **Specialist dementia nurses:** Provide practical, clinical, and emotional support.
* **Equipment and adaptations:** Can make the home environment safer and more manageable. (See factsheet 429)
* **Activities handbook:** Provides ideas for engaging activities. (See booklet 77AC)
* **Keeping active and involved:** Important for physical and mental well-being. (See booklet 1506)

**In the Community:**

* **Local dementia advisers and support workers:** Offer advice, information, and practical support.
* **Day centres:** Provide activities,